### Setting the environment

In [1]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import os 
os.chdir('/content/drive/MyDrive/NEREL/Cross_Domain_Nested_NER')

In [3]:
!pip install fastNLP==1.0.1 -q
!pip install transformers -q
!pip install sparse==0.13.0 -q
!pip install torch==1.11.0+cu113 --extra-index-url https://download.pytorch.org/whl/cu113 -q
!pip install llvmlite==0.38.1 --user -q
!pip install numba==0.55.2 --user -q

In [4]:
import torch
print(torch.__version__)

In [5]:
!nvcc --version

In [6]:
!pip install torch-scatter --no-index -f https://data.pyg.org/whl/torch-{torch.__version__}.html 

In [7]:
import json
import warnings
import argparse
import collections

if 'p' in os.environ:
    os.environ['CUDA_VISIBLE_DEVICES'] = os.environ['p']
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['MKL_THREADING_LAYER'] = 'GNU'
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import fastNLP
from fastNLP import cache_results, prepare_torch_dataloader
from fastNLP import Evaluator
from fastNLP import SortedSampler, BucketedBatchSampler

from model.model import CNNNer
from model.metrics import NERMetric
from preprocessing.ner_pipe import SpanNerPipe
from preprocessing.padder import Torch3DMatrixPadder
from model.metrics_utils import decode
from fastNLP import print as pr
from tqdm.notebook import tqdm


seed_value=42
np.random.seed(seed_value)
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)
os.environ['FASTNLP_GLOBAL_SEED'] = str(seed_value)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
def calculate(path, count=False):
    max_sent_len = 0
    total_lengths = 0
    total_ent_length = 0
    max_ent_length = 0
    num_ents = 0
    overlapped_ent_num = 0
    num_sent = 0
    counts = {}
    with open(path, 'r') as f:
        for line in f:
            num_sent += 1
            data = json.loads(line.strip())
            max_sent_len = max(max_sent_len, len(data['tokens']))
            total_lengths += len(data['tokens'])
            flags = [0]*len(data['tokens'])
            for ent in data['entity_mentions']:
                num_ents += 1
                if ent['entity_type'] not in counts.keys():
                    counts[ent['entity_type']] = 1
                else:
                    counts[ent['entity_type']] += 1
                start, end = ent['start'], ent['end']
                total_ent_length += ent['end'] - ent['start']
                max_ent_length = max(max_ent_length, ent['end'] - ent['start'])
                for i in range(start, end):
                    flags[i] += 1
            for ent in data['entity_mentions']:
                start, end = ent['start'], ent['end']
                if any([flags[i]>1 for i in range(start, end)]):
                    overlapped_ent_num += 1

    pr(f"For {path}")
    pr("total sentence ", num_sent)
    pr("average sentence length ", total_lengths/num_sent)
    pr("max sentence length ", max_sent_len)

    pr('num_entities ', num_ents)
    pr('average entity length  ', total_ent_length/num_ents)
    pr('max entity length  ', max_ent_length)
    pr("Number of nested entity ", overlapped_ent_num)

    pr("Number of tokens ", total_lengths)
    pr()
    if count:
        return counts

def get_stats(directory): 
    for name in ['train', 'dev', 'test']:
        path = f'{directory}/{name}.jsonl'
        if name == 'test':
            counts = calculate(path, count=True)
        else:
            calculate(path)
    return counts

In [9]:
counts_nerel = get_stats('../nlp_project/datasets/outputs/nerel_common_labels')

In [10]:
counts_nerel_bio = get_stats('../nlp_project/datasets/outputs/nerel_bio_common_labels')

### Fetching and preprocessing the dataset

In [11]:
###### HYPERPARAMS ######
lr = 5e-6
batch_size = 4
n_epochs = 10
warmup = 0.1
model_name = 'DeepPavlov/rubert-base-cased'
# model_name = 'cointegrated/rubert-tiny2'
cnn_depth = 3
cnn_dim = 100
logit_drop = 0
biaffine_size = 200
n_head = 4
accumulation_steps = 1
########################
non_ptm_lr_ratio = 100
schedule = 'linear'
weight_decay = 1e-2
size_embed_dim = 25
ent_thres = 0.5
kernel_size = 3
########################

def get_data(dataset_name, model_name):
    paths = f'../nlp_project/datasets/outputs/{dataset_name}'
    pipe = SpanNerPipe(model_name=model_name)
    dl = pipe.process_from_file(paths)
    return dl, pipe.matrix_segs

def densify(x):
    x = x.todense().astype(np.float32)
    return x
    
def preprocess_data(dataset_name, model_name):

    dl, matrix_segs = get_data(dataset_name, model_name)

    dl.apply_field(densify, field_name='matrix', new_field_name='matrix', progress_bar='Densify')

    print(dl)

    label2idx = getattr(dl, 'ner_vocab') if hasattr(dl, 'ner_vocab') else getattr(dl, 'label2idx')
    print(f"{len(label2idx)} labels: {label2idx}, matrix_segs:{matrix_segs}")

    dls = {}
    for name, ds in dl.iter_datasets():
        ds.set_pad('matrix', pad_fn=Torch3DMatrixPadder(pad_val=ds.collator.input_fields['matrix']['pad_val'],
                                                        num_class=matrix_segs['ent'],
                                                        batch_size=batch_size))

        if name in ['train', 'dev']:
            _dl = prepare_torch_dataloader(ds, batch_size=batch_size, num_workers=0,
                                           batch_sampler=BucketedBatchSampler(ds, 'input_ids',
                                                                              batch_size=batch_size,
                                                                              num_batch_per_bucket=30),
                                           pin_memory=True, shuffle=True)
        else:
            _dl = prepare_torch_dataloader(ds, batch_size=1, num_workers=0, pin_memory=True, shuffle=False)

        dls[name] = _dl
    
    return dls, label2idx, matrix_segs

In [12]:
dls_nerel, label2idx_nerel, matrix_segs_nerel = preprocess_data('nerel_common_labels', model_name)

In [13]:
dls_nerel_bio, label2idx_nerel_bio, matrix_segs_nerel_bio = preprocess_data('nerel_bio_common_labels', model_name)

### Loading and evaluating the models

In [14]:
model_nerel_1 = CNNNer(model_name, num_ner_tag=matrix_segs_nerel['ent'], cnn_dim=cnn_dim, biaffine_size=biaffine_size,
               size_embed_dim=size_embed_dim, logit_drop=logit_drop,
               kernel_size=kernel_size, n_head=n_head, cnn_depth=cnn_depth)

model_nerel_1 = torch.load('../nlp_project/weights/model_best_common_labels_bertbase_cnn_8965.pkl')

counter = collections.Counter()
for name, param in model_nerel_1.named_parameters():
    counter[name.split('.')[0]] += torch.numel(param)
print(counter)
print("Total param ", sum(counter.values()))

In [15]:
# model_nerel_2 = CNNNer(model_name, num_ner_tag=matrix_segs_nerel_bio['ent'], cnn_dim=cnn_dim, biaffine_size=biaffine_size,
#                size_embed_dim=size_embed_dim, logit_drop=logit_drop,
#                kernel_size=kernel_size, n_head=n_head, cnn_depth=cnn_depth)

# model_nerel_2 = torch.load('../nlp_project/weights/model_best_common_labels_berttiny_cnn_8036.pkl')

# counter = collections.Counter()
# for name, param in model_nerel_2.named_parameters():
#     counter[name.split('.')[0]] += torch.numel(param)
# print(counter)
# print("Total param ", sum(counter.values()))

In [16]:
allow_nested = True
metrics = {'ner': NERMetric(matrix_segs=matrix_segs_nerel, ent_thres=ent_thres, allow_nested=allow_nested)}

In [17]:
tester1 = fastNLP.Evaluator(model=model_nerel_1, driver='torch', dataloaders=dls_nerel['test'], metrics=metrics, device=0)
tester1.run()

In [18]:
tester2 = fastNLP.Evaluator(model=model_nerel_1, driver='torch', dataloaders=dls_nerel_bio['test'], metrics=metrics, device=0)
tester2.run()

### Getting predictions

In [19]:
from model.metrics_utils import decode

# given dataloader and model, return all predictions, sentence lengths, tokens and targets
def predict(dataloader, model):
    model.eval()
    outputs_list = []
    wordlen_list = []
    tokens_list = []
    targets_list = []
    with torch.no_grad():
        for i, item in enumerate(tqdm(dataloader)):
            inputs = item['input_ids'].to(device)
            indexes = item['indexes'].to(device)
            bpe_len = item['bpe_len'].to(device)
            word_len = item['word_len'].to(device)
            matrix = item['matrix'].to(device)
            tokens = item['tokens']
            target = item['ent_target']
            outputs = model(inputs, bpe_len, indexes, matrix)
            outputs_list.append(outputs['scores'])
            wordlen_list.append(word_len)
            tokens_list.append(tokens)
            targets_list.append(target)
    return outputs_list, wordlen_list, tokens_list, targets_list

# util function
def get_key_by_value(dictionary, value):
    return list(dictionary.keys())[list(dictionary.values()).index(value)]

# converting integer labels to string labels
def labels_to_str(pred_list, label2idx):
    pred_list_str = [None]*len(pred_list)

    for i in range(len(pred_list)):
        for s, e, ent_type in pred_list[i]:
            if pred_list_str[i] is not None:
                pred_list_str[i].append([s, e+1, get_key_by_value(label2idx, ent_type)])
            else:
                pred_list_str[i] = [[s, e+1, get_key_by_value(label2idx, ent_type)]]
        if pred_list_str[i] is None:
            pred_list_str[i] = []
    return pred_list_str
    
# decode score matrix into actual predictions
def decode_outputs(scores, word_lens):

    span_ids = [None]*len(scores)
    pred_list = [None]*len(scores)

    for i in range(len(scores)):
        # getting score matrix containing probabilities vector for each span cell
        scores[i] = scores[i].sigmoid()
        scores[i] = (scores[i] + scores[i].transpose(1, 2))/2
        # getting valid spans from score matrix based on max probability in prob vector
        span_ids[i] = decode(scores[i].max(dim=-1)[0], word_lens[i], allow_nested=True, thres=0.5)

        batch_preds = []
        for span_id, pred_score in zip(span_ids[i], scores[i].cpu().numpy()):
            temp_preds = set()
            for s, e, l in span_id:
                score = pred_score[s, e]
                ent_type = score.argmax()
                if score[ent_type]>=0.5:
                    temp_preds.add((s, e, ent_type))

            batch_preds.append(list(temp_preds))

        pred_list[i] = batch_preds[0]   
    
    return pred_list

### Calculating metrics per each category

In [20]:
def split_by_label(preds, targets, label2idx):
    ppl = {key:set() for key in range(len(label2idx))}
    tpl = {key:set() for key in range(len(label2idx))}

    for i in range(len(preds)):
        cur_target = set(map(tuple, targets[i]))
        cur_pred = set(map(tuple, preds[i]))
        for item in cur_pred:
            ppl[item[2]].add(item)
        for item in cur_target:
            tpl[item[2]].add(item)
    return ppl, tpl


def metrics_to_str(f1, pre, rec, label2idx):
    f1_str, pre_str, rec_str = {}, {}, {}

    for i in range(len(f1)):
#         if pre[i] != 0.0 else '-'
        f1_str[get_key_by_value(label2idx, i)] = f1[i]
        pre_str[get_key_by_value(label2idx, i)] = pre[i] 
        rec_str[get_key_by_value(label2idx, i)] = rec[i]
        
    return f1_str, pre_str, rec_str


def metrics_by_label(ppl, tpl, str_label=False, label2idx=None):
    tp = {key:0 for key in range(len(ppl))}
    pre = {key:0 for key in range(len(ppl))}
    rec = {key:0 for key in range(len(ppl))}
    f1 = {key:0 for key in range(len(ppl))}
    
    for i in range(len(ppl)):
        tp[i] += len(tpl[i].intersection(ppl[i]))
        pre[i] += len(ppl[i])
        rec[i] += len(tpl[i])
    
    for i in range(len(ppl)):
        pre[i] = tp[i]/(pre[i]+1e-6)
        rec[i] = tp[i]/(rec[i]+1e-6)
        f1[i] = 2*pre[i]*rec[i]/(pre[i]+rec[i]+1e-6)
        
        f1[i] = round(f1[i] * 100, 2)
        pre[i] = round(pre[i] * 100, 2)
        rec[i] = round(rec[i] * 100, 2)
    
    if str_label:
        f1_str, pre_str, rec_str = metrics_to_str(f1, pre, rec, label2idx)
        return f1_str, pre_str, rec_str
    else:
        return f1, pre, rec


def metrics_micro(pred_list, targets):
    tp = 0
    pre = 0
    rec = 0
    for pred, target in zip(pred_list, targets):
        cur_target = set(map(tuple, target))
        tp += len(cur_target.intersection(pred))
        pre += len(pred)
        rec += len(cur_target)
        
    pre = tp/(pre+1e-6)
    rec = tp/(rec+1e-6)
    f1 = 2*pre*rec/(pre+rec+1e-6)
    
    return f1, pre, rec

def squeeze_batch_dim(input_list):
    res = input_list
    for i in range(len(input_list)):
        res[i] = res[i][0]
    return res

In [21]:
# predictiong on NEREL using NEREL model
scores_nerel, word_lens_nerel, tokens_nerel, targets_nerel = predict(dls_nerel['test'], model_nerel_1)

tokens_nerel = squeeze_batch_dim(tokens_nerel)
targets_nerel = squeeze_batch_dim(targets_nerel)

pred_list_nerel = decode_outputs(scores_nerel, word_lens_nerel)

preds_per_label_nerel, targets_per_label_nerel = split_by_label(pred_list_nerel, targets_nerel, label2idx_nerel)
f1_score_nerel, precision_nerel, recall_nerel = metrics_by_label(preds_per_label_nerel, targets_per_label_nerel, str_label=True, label2idx=label2idx_nerel)

# predicting on NEREL-BIO using NEREL model
scores_nerel_bio, word_lens_nerel_bio, tokens_nerel_bio, targets_nerel_bio = predict(dls_nerel_bio['test'], model_nerel_1)

tokens_nerel_bio = squeeze_batch_dim(tokens_nerel_bio)
targets_nerel_bio = squeeze_batch_dim(targets_nerel_bio)

pred_list_nerel_bio = decode_outputs(scores_nerel_bio, word_lens_nerel_bio)

preds_per_label_nerel_bio, targets_per_label_nerel_bio = split_by_label(pred_list_nerel_bio, targets_nerel_bio, label2idx_nerel_bio)
f1_score_nerel_bio, precision_nerel_bio, recall_nerel_bio = metrics_by_label(preds_per_label_nerel_bio, targets_per_label_nerel_bio, str_label=True, label2idx=label2idx_nerel_bio)

In [22]:
common_labels = ['AGE', 'CITY', 'COUNTRY', 'DATE', 'DISO', 'DISEASE', 'FACILITY', 'LOCATION', 'NUMBER', 'ORDINAL', 'ORGANIZATION', 'PERCENT', 'PERSON', 'PRODUCT', 'PROFESSION', 'STATE_OR_PROVINCE', 'TIME']

In [23]:
metrics_nerel = {'f1': list(f1_score_nerel.values()), 'precision': list(precision_nerel.values()), 'recall': list(recall_nerel.values())}

df_metrics_nerel = pd.DataFrame(data=metrics_nerel, index=list(f1_score_nerel.keys()))

counts_nerel = dict(sorted(counts_nerel.items()))
df_metrics_nerel['counts'] = list(counts_nerel.values())

df_metrics_nerel = df_metrics_nerel[df_metrics_nerel.index.isin(common_labels)]
df_metrics_nerel

In [24]:
metrics_nerel_bio = {'f1': list(f1_score_nerel_bio.values()), 'precision': list(precision_nerel_bio.values()), 'recall': list(recall_nerel_bio.values())}


df_metrics_nerel_bio = pd.DataFrame(data=metrics_nerel_bio, index=list(f1_score_nerel_bio.keys()))

# counts_nerel_bio['GENE'] = 0
counts_nerel_bio = dict(sorted(counts_nerel_bio.items()))
df_metrics_nerel_bio['counts'] = list(counts_nerel_bio.values())

# df_metrics_nerel_bio = df_metrics_nerel_bio[df_metrics_nerel_bio.index.isin(common_labels)]
# df_metrics_nerel_bio = df_metrics_nerel_bio.rename(index={'DISO': 'DISEASE'})
df_metrics_nerel_bio

### Rendering and analyzing predictions using SpaCy

In [25]:
from spacy import displacy
from pathlib import Path
import random

def get_colors(labels): 
    colors = dict.fromkeys(labels)
    for key in colors.keys():
        rand = lambda: random.randint(100, 220)
        colors[key] = '#%02X%02X%02X' % (rand(), rand(), rand())
    return colors

def render_one_pred(inputs, preds, toFile=False, colors=None, filename=None):
    span_dict = {}
    span_dict["tokens"] = inputs
    span_dict["spans"] = []
    if preds:
        for i in range(len(preds)): 
            start = preds[i][0]
            end = preds[i][1]
            ent = preds[i][2]
            span_dict["spans"].append({"start_token": start, "end_token": end, "label": ent})
            
    if colors is None:
        colors = {'AGE': '#AC9D90', 'AWARD': '#CD6777', 'CITY': '#758B73', 'COUNTRY': '#AC6479', 'CRIME': '#72D5D6',
                  'DATE': '#CAAE9C', 'DISEASE': '#65B8C2', 'DISTRICT': '#BC64B1', 'EVENT': '#CA8F75', 'FACILITY': '#6DA283', 
                  'FAMILY': '#7688D8', 'IDEOLOGY': '#B9B072', 'LANGUAGE': '#919E96', 'LAW': '#7CA6CC', 'LOCATION': '#D5C1B1', 
                  'MONEY': '#926F6E', 'NATIONALITY': '#708DD8', 'NUMBER': '#AF75CB', 'ORDINAL': '#C76585', 'ORGANIZATION': '#7394BA',
                  'PENALTY': '#948A67', 'PERCENT': '#729384', 'PERSON': '#B2DABF', 'PRODUCT': '#CFA4C1', 'PROFESSION': '#968870', 
                  'RELIGION': '#90AEA4', 'STATE_OR_PROVINCE': '#A295A3', 'TIME': '#BCC89B', 'WORK_OF_ART': '#D9C187'}
    
    options = {"colors": colors, "compact": False}
    if toFile:
        svg = displacy.render(span_dict, style="span", manual=True, page=True, jupyter=False, options=options)
        if filename is not None:
            output_path = Path(f"{filename}.html")
        else:
            output_path = Path("no_filename.html")
        output_path.open("w", encoding="utf-8").write(svg)
    else:
        displacy.render(span_dict, style="span", manual=True, jupyter=True, options=options)
        
def render_all_preds(inputs, preds):
    for i in range(len(inputs)):
        render_one_pred(inputs[i], preds[i], toFile=False)
        print('-------------------------------------------------------------------------------------------------------------------------')
        
        
def filter_labels(annotations, keep):
    new_annotations = []
    for anns in annotations:
        temp = list(filter(lambda ann: ann[2] in keep, anns))
        new_annotations.append(temp)
    return new_annotations

In [26]:
pred_list_nerel_str = labels_to_str(pred_list_nerel, label2idx_nerel)
targets_nerel_str = labels_to_str(targets_nerel, label2idx_nerel)

# render_all_preds(tokens_nerel, pred_list_nerel_str)

In [27]:
pred_list_nerel_bio_str = labels_to_str(pred_list_nerel_bio, label2idx_nerel_bio)
targets_nerel_bio_str = labels_to_str(targets_nerel_bio, label2idx_nerel)

# render_all_preds(tokens_nerel_bio, pred_list_nerel_bio_str)

In [28]:
def render_one_comparison(tokens, preds, targets, toFile=False, filename='none'):
    if toFile == True:
        pred_filename = filename + '_pred'
        target_filename = filename + '_target'
    else:
        pred_filename = None
        target_filename = None
    print('--------------------------------------------------- PREDICTED ANNOTATION ----------------------------------------------------')
    render_one_pred(tokens, preds, toFile=toFile, filename=pred_filename)
    print('---------------------------------------------------- TARGET ANNOTATION ------------------------------------------------------')
    render_one_pred(tokens, targets, toFile=toFile, filename=target_filename)
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print()                         
    print()

In [29]:
def compare_preds_targets(all_tokens, all_preds, all_targets, error='FN'):
    j = 0
    for tokens, preds, targets in zip(all_tokens, all_preds, all_targets):
        preds = list(map(tuple, preds))
        targets = list(map(tuple, targets))
        
        if error == 'FN':
            if len(preds) < len(targets): # and ('DISEASE' in [entry[2] for entry in preds]) or ('DISEASE' in [entry[2] for entry in targets]):
                print(j)
                render_one_comparison(tokens, preds, targets, toFile=False)   
                
        elif error == 'FP':
            if len(preds) > len(targets): # and ('DISEASE' in [entry[2] for entry in preds]) or ('DISEASE' in [entry[2] for entry in targets]):
                print(j)
                render_one_comparison(tokens, preds, targets, toFile=False)         
        else:
            if sorted(preds) != sorted(targets):
                if len(preds) == len(targets): 
                    print(j)
                    render_one_comparison(tokens, preds, targets, toFile=False)
        j += 1

### NEREL-C

In [30]:
# FN
print(' '.join(tokens_nerel[468]))
print(pred_list_nerel_str[468])
print(targets_nerel_str[468])

render_one_comparison(tokens_nerel[468], pred_list_nerel_str[468], targets_nerel_str[468])

In [31]:
# FP

print(' '.join(tokens_nerel[8]))
print(pred_list_nerel_str[8])
print(targets_nerel_str[8])
      
render_one_comparison(tokens_nerel[8], pred_list_nerel_str[8], targets_nerel_str[8])

In [32]:
# CLOS

print(' '.join(tokens_nerel[211]))
print(pred_list_nerel_str[211])
print(targets_nerel_str[211])
      
render_one_comparison(tokens_nerel[211], pred_list_nerel_str[211], targets_nerel_str[211])
render_one_comparison(tokens_nerel[318], pred_list_nerel_str[318], targets_nerel_str[318], toFile=False, filename=None)
render_one_comparison(tokens_nerel[370], pred_list_nerel_str[370], targets_nerel_str[370], toFile=False, filename=None)

In [33]:
# WLCS

print(' '.join(tokens_nerel[11]))
print(pred_list_nerel_str[11])
print(targets_nerel_str[11])
      
render_one_comparison(tokens_nerel[11], pred_list_nerel_str[11], targets_nerel_str[11])

In [34]:
# Triple WLCS 

print(' '.join(tokens_nerel[358]))
print(pred_list_nerel_str[358])
print(targets_nerel_str[358])
      
render_one_comparison(tokens_nerel[358], pred_list_nerel_str[358], targets_nerel_str[358])

In [35]:
# ERROR IN ANNOTATION and FN

print(' '.join(tokens_nerel[423]))
print(pred_list_nerel_str[423])
print(targets_nerel_str[423])
      
render_one_comparison(tokens_nerel[423], pred_list_nerel_str[423], targets_nerel_str[423])

In [36]:
# QUESTIONABLE (CLOS) or (FP)

print(' '.join(tokens_nerel[308]))
print(pred_list_nerel_str[308])
print(targets_nerel_str[308])
      
render_one_comparison(tokens_nerel[308], pred_list_nerel_str[308], targets_nerel_str[308])

In [37]:
# WLOS

print(' '.join(tokens_nerel[420]))
print(pred_list_nerel_str[420])
print(targets_nerel_str[420])
      
render_one_comparison(tokens_nerel[420], pred_list_nerel_str[420], targets_nerel_str[420])
render_one_comparison(tokens_nerel[149], pred_list_nerel_str[149], targets_nerel_str[149])

In [38]:
# compare_preds_targets(tokens_nerel, pred_list_nerel_str, targets_nerel_str, error=None)

### NEREL-BIO-C

In [39]:
# FN

print(' '.join(tokens_nerel_bio[41]))
print(pred_list_nerel_bio_str[41])
print(targets_nerel_bio_str[41])
      
render_one_comparison(tokens_nerel_bio[41], pred_list_nerel_bio_str[41], targets_nerel_bio_str[41])

In [40]:
# FP

print(' '.join(tokens_nerel_bio[635]))
print(pred_list_nerel_bio_str[635])
print(targets_nerel_bio_str[635])
      
render_one_comparison(tokens_nerel_bio[635], pred_list_nerel_bio_str[635], targets_nerel_bio_str[635])

In [41]:
# WLOS

print(' '.join(tokens_nerel_bio[622]))
print(pred_list_nerel_bio_str[622])
print(targets_nerel_bio_str[622])
      
render_one_comparison(tokens_nerel_bio[622], pred_list_nerel_bio_str[622], targets_nerel_bio_str[622])

In [42]:
# WLCS

print(' '.join(tokens_nerel_bio[115]))
print(pred_list_nerel_bio_str[115])
print(targets_nerel_bio_str[115])
      
render_one_comparison(tokens_nerel_bio[115], pred_list_nerel_bio_str[115], targets_nerel_bio_str[115])
render_one_comparison(tokens_nerel_bio[179], pred_list_nerel_bio_str[179], targets_nerel_bio_str[179])

In [43]:
# CLOS

print(' '.join(tokens_nerel_bio[599]))
print(pred_list_nerel_bio_str[599])
print(targets_nerel_bio_str[599])
      
render_one_comparison(tokens_nerel_bio[599], pred_list_nerel_bio_str[599], targets_nerel_bio_str[599])

In [44]:
# QUESTIONABLE (CLOS or FP)

print(' '.join(tokens_nerel_bio[125]))
print(pred_list_nerel_bio_str[125])
print(targets_nerel_bio_str[125])
      
render_one_comparison(tokens_nerel_bio[125], pred_list_nerel_bio_str[125], targets_nerel_bio_str[125])

In [45]:
# ERROR IN ANNOTATION

print(' '.join(tokens_nerel_bio[186]))
print(pred_list_nerel_bio_str[186])
print(targets_nerel_bio_str[186])
      
render_one_comparison(tokens_nerel_bio[186], pred_list_nerel_bio_str[186], targets_nerel_bio_str[186])

In [46]:
# compare_preds_targets(tokens_nerel_bio, pred_list_nerel_bio_str, targets_nerel_bio_str, error=None)